In [1]:
from config import Config
from game import Game
import numpy as np
from time import time

2024-02-08 15:43:05.347255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Image generation times

- Optimization #1: Cache image for root node during MCTS. Remaking same image in training data generation is pointless.
- Optimization #2: Similar concept can be used for picked move.
- Optimization #3: Not generating a new image each time but updating the previous one.

In [2]:
config = Config()
config.max_game_length = 200
times = []
while len(times) < 1000:
    game = Game(config)
    while not game.terminal():
        start = time()
        game.make_image(-1)
        end = time()
        times.append(end - start)
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times) == 1000:
            break
avg_img_gen_time = np.mean(times)

In [18]:
# WARN: Data does not asume tree reuse, so it is not 100% accurate. Also terminal nodes are not expanded. Actual time will be lower than this.
# Image generation happens on each selected move and during playouts when expanding a leaf node
num_simulations = config.num_mcts_sims[1] * 0.25 + config.num_mcts_sims[0] * 0.75
#num_simulations = int(num_simulations) # Average number of simulations per move in a game (assuming 25% of simulations are full searches)
num_simulations = 100
avg_moves_per_game = 100
num_image_generations = avg_moves_per_game * (num_simulations + 1) + avg_moves_per_game # One image per simulation, N sims per move, plus one image per move (generate training data)
avg_img_gen_time_per_game1 = avg_img_gen_time * num_image_generations
print(f"Average image generation time: {avg_img_gen_time}s per image")
print(f"Average image generation time per game (including train data gen): {avg_img_gen_time_per_game1}s")

num_image_generations = avg_moves_per_game * (num_simulations + 1) # + 1 for root node (Not counted in num_simulations but still expanded)
avg_img_gen_time_per_game2 = avg_img_gen_time * num_image_generations
print(f"Average image generation time per game (excluding train data gen): {avg_img_gen_time_per_game2}s")

# Calc time saved by caching root image
avg_img_time_diff = np.abs(avg_img_gen_time_per_game1 - avg_img_gen_time_per_game2)
# Per 100 moves
print(f"Average image generation time difference: {avg_img_time_diff}s per {avg_moves_per_game} moves")
# Per 1 training data set (10000 samples)
print(f"Average image generation time difference: {avg_img_time_diff * 100}s per 10000 training samples")
# Per 1 training session (500000 samples)
avg_time_diff_s = avg_img_time_diff * 5000
print(f"Average image generation time difference: {avg_time_diff_s}s per 500000 training samples")
avg_time_diff_m = avg_time_diff_s / 60
print(f"Average image generation time difference: {avg_time_diff_m}m per 500000 training samples")
print("All calculation above * 2 if optimization #2 is also implemented")

Average image generation time: 0.0043626298904418945s per image
Average image generation time per game (including train data gen): 44.49882488250732s
Average image generation time per game (excluding train data gen): 44.06256189346313s
Average image generation time difference: 0.43626298904418803s per 100 moves
Average image generation time difference: 43.6262989044188s per 10000 training samples
Average image generation time difference: 2181.31494522094s per 500000 training samples
Average image generation time difference: 36.355249087015665m per 500000 training samples
All calculation above * 2 if optimization #2 is also implemented


In [4]:
# Highly Theoretical calculation for optimization #3
num_image_generations = avg_moves_per_game * (num_simulations) # Including optimization #1 & #2
num_img_updates = avg_moves_per_game * (num_simulations - 1)
theoretical_perc_time_gain_per_image_min = 0.01
theoretical_time_gain_per_image = avg_img_gen_time * theoretical_perc_time_gain_per_image_min # 1% time gain per image
theoretical_time_gain1 = theoretical_time_gain_per_image * num_img_updates
print(f"Theoretical time gain: {theoretical_time_gain1}s per 100 moves for {theoretical_perc_time_gain_per_image_min*100}% time gain per image update")
theoretical_perc_time_gain_per_image_mid = 0.1
theoretical_time_gain_per_image = avg_img_gen_time * theoretical_perc_time_gain_per_image_mid # 1% time gain per image
theoretical_time_gain2 = theoretical_time_gain_per_image * num_img_updates
print(f"Theoretical time gain: {theoretical_time_gain2}s per 100 moves for {theoretical_perc_time_gain_per_image_mid*100}% time gain per image update")
theoretical_perc_time_gain_per_image_max = 0.25
theoretical_time_gain_per_image = avg_img_gen_time * theoretical_perc_time_gain_per_image_max # 1% time gain per image
theoretical_time_gain3 = theoretical_time_gain_per_image * num_img_updates
print(f"Theoretical time gain: {theoretical_time_gain3}s per 100 moves for {theoretical_perc_time_gain_per_image_max*100}% time gain per image update")
per_10000_training_samples_min = theoretical_time_gain1 * 100
per_10000_training_samples_max = theoretical_time_gain3 * 100
print(f"Time gain per 10000 samples between {per_10000_training_samples_min}s and {per_10000_training_samples_max}s")
print(f"With 6 actors, time gain per 10000 samples between {per_10000_training_samples_min/6/60}m and {per_10000_training_samples_max/6/60}m")

Theoretical time gain: 0.5845897536277771s per 100 moves for 1.0% time gain per image update
Theoretical time gain: 5.845897536277772s per 100 moves for 10.0% time gain per image update
Theoretical time gain: 14.614743840694429s per 100 moves for 25.0% time gain per image update
Time gain per 10000 samples between 58.45897536277771s and 1461.4743840694427s
With 6 actors, time gain per 10000 samples between 0.1623860426743825m and 4.059651066859563m


In [5]:
# Optimistic worst case scenario for all three optimizations
print("Calculations asume 6 agents generating training data")
total_time_gain_per_training_session = per_10000_training_samples_min * 500 + avg_time_diff_s * 2
print(f"Worst case scenario with all optimizations. Image update % time gain: {theoretical_perc_time_gain_per_image_min*100}%")
print(f"Total time gain per 500000 samples: {total_time_gain_per_training_session}s")
print(f"Total time gain per 500000 samples: {total_time_gain_per_training_session/60/6}m")
print(f"Total time gain per 500000 samples: {total_time_gain_per_training_session/60/60/6}h")

total_time_gain_per_training_session = per_10000_training_samples_max * 500 + avg_time_diff_s * 2
print(f"Best case scenario with all optimizations. Image update % time gain: {theoretical_perc_time_gain_per_image_max*100}%")
print(f"Total time gain per 500000 samples: {total_time_gain_per_training_session}s")
print(f"Total time gain per 500000 samples: {total_time_gain_per_training_session/60/6}m")
print(f"Total time gain per 500000 samples: {total_time_gain_per_training_session/60/60/6}h")

Calculations asume 6 agents generating training data
Worst case scenario with all optimizations. Image update % time gain: 1.0%
Total time gain per 500000 samples: 33592.09778308871s
Total time gain per 500000 samples: 93.31138273080198m
Total time gain per 500000 samples: 1.5551897121800329h
Best case scenario with all optimizations. Image update % time gain: 25.0%
Total time gain per 500000 samples: 735099.8021364212s
Total time gain per 500000 samples: 2041.9438948233922m
Total time gain per 500000 samples: 34.03239824705654h


#### Create image rewrite tests

In [3]:
from gameimage import board_to_image
#from gameimage_v2 import board_to_image as board_to_image_v2
num_times = 1000
config = Config()
config.max_game_length = 200
""" times = []
while len(times) < num_times:
    game = Game(config)
    while not game.terminal():
        start = time()
        board_to_image(game.board)
        end = time()
        times.append(end - start)
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times) == num_times:
            break
avg_img_gen_time = np.mean(times) """

times2 = []
while len(times2) < num_times:
    game = Game(config)
    while not game.terminal():
        start = time()
        board_to_image(game.board)
        end = time()
        times2.append(end - start)
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times2) == num_times:
            break
avg_img_gen_time2 = np.mean(times2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at /home/tomaz/Documents/ChessBot_2.13/main/gameimage/__init__.py (72)[0m
[1m
File "gameimage/__init__.py", line 72:[0m
[1mdef get_castlingrights_planes(board: chess.Board):
    <source elided>

[1m@njit
[0m[1m^[0m[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'chess.Board'>[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'chess.Board'>[0m


In [21]:
print(f"Average image generation time (old): {avg_img_gen_time}s per image")
print(f"Average image generation time (rewrite): {avg_img_gen_time2}s per image")

Average image generation time (old): 0.004468789577484131s per image
Average image generation time (rewrite): 0.0038433365821838377s per image


##### Actual image update tests

In [13]:
from gameimage_v2 import board_to_image as board_to_image_v2
from gameimage_v2 import update_image
num_times = 10000
config = Config()
config.max_game_length = 200
times = []
while len(times) < num_times:
    game = Game(config)
    image = None
    while not game.terminal():
        start = time()
        if image is None:
            image = board_to_image_v2(game.board)
        else:
            image = update_image(game.board, image)
        end = time()
        times.append(end - start)
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times) == num_times:
            break
avg_img_update_time = np.mean(times)

In [14]:
print(f"Average image generation time (rewrite + update): {avg_img_update_time}s per image")

Average image generation time (rewrite + update): 0.0005570780754089355s per image


##### Actual image update tests with cython

In [16]:

import test_cython
num_times = 10000
config = Config()
config.max_game_length = 200
times = []
while len(times) < num_times:
    game = Game(config)
    image = None
    while not game.terminal():
        start = time()
        if image is None:
            image = test_cython.board_to_image(game.board)
        else:
            image = test_cython.update_image(game.board, image)
        end = time()
        times.append(end - start)
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times) == num_times:
            break
    image = None
avg_img_update_time2 = np.mean(times)
print(f"Average image generation time (rewrite + update + cython): {avg_img_update_time2}s per image")

Average image generation time (rewrite + update + cython): 0.0004755209922790527s per image


In [17]:
total_img_update_time2 = np.sum(times) / 100
print(f"Total image generation time (rewrite + update + cython): {total_img_update_time2}s per 100 moves")


Total image generation time (rewrite + update + cython): 0.04755209922790527s per 100 moves


#### UCI move to action times

- Optimization #1: Create a map for each player of mapping uci moves to actions.
- Optimization #2: If above doesn't work. Cache during uci to action mapping during self play. No need to recalc after first calculation.

In [8]:
from actionspace.mapper import uci_to_action
config = Config()
config.max_game_length = 200
times = []
while len(times) < 1000:
    game = Game(config)
    while not game.terminal():
        move = np.random.choice(game.legal_moves())
        start = time()
        action = uci_to_action(move, game.to_play())
        end = time()
        game.make_move(move)
        times.append(end - start)
        if len(times) == 1000:
            break
avg_utoa_call_time = np.mean(times)

In [9]:
# WARN: Data does not asume tree reuse, so it is not 100% accurate. Also terminal nodes are not expanded. Actual time will be lower than this.
# We can use same simulation numbers as above
# On average there are about 30 possible moves per turn
avg_num_moves_per_position = 30
# uci_to_action is called on each possible move during playouts when expanding a leaf node and when generating training data
num_utoa_calls = avg_moves_per_game * (num_simulations + 1) * avg_num_moves_per_position #  During MCTS on end of each simulation
print(f"Average uci_to_action call time: {avg_utoa_call_time}s per call")
avg_utoa_call_time_per_game = avg_utoa_call_time * num_utoa_calls
print(f"Average uci_to_action call time per game (excluding train data generation): {avg_utoa_call_time_per_game}s")
num_utoa_calls += avg_moves_per_game * avg_num_moves_per_position
avg_utoa_call_time_per_game = avg_utoa_call_time * num_utoa_calls
print(f"Average uci_to_action call time per game (including train data generation): {avg_utoa_call_time_per_game}s")

Average uci_to_action call time: 0.00010584878921508789 s per call
Average uci_to_action call time per game (excluding train data generation): 43.18630599975586 s
Average uci_to_action call time per game (including train data generation): 43.50385236740112 s


In [10]:
# Theoretically we could have a cached map of uci to action
action_space_map = {i: i for i in range(4672)}
config = Config()
config.max_game_length = 200
times = []
while len(times) < 1000:
    game = Game(config)
    while not game.terminal():
        move = np.random.choice(game.legal_moves())
        action = uci_to_action(move, game.to_play())
        start = time()
        action = action_space_map[action]
        end = time()
        game.make_move(move)
        times.append(end - start)
        if len(times) == 1000:
            break
avg_utoa_cached_call_time = np.mean(times)

In [11]:
# uci_to_action is called on each possible move during playouts when expanding a leaf node and when generating training data
num_utoa_calls = avg_moves_per_game * (num_simulations + 1) * avg_num_moves_per_position #  During MCTS on end of each simulation
print(f"Average uci_to_action call time: {avg_utoa_cached_call_time}s per call")
avg_utoa_cached_call_time_per_game = avg_utoa_cached_call_time * num_utoa_calls
print(f"Average uci_to_action call time per game (excluding train data generation): {avg_utoa_cached_call_time_per_game}s")
num_utoa_calls += avg_moves_per_game * avg_num_moves_per_position
avg_utoa_cached_call_time_per_game = avg_utoa_cached_call_time * num_utoa_calls
print(f"Average uci_to_action call time per game (including train data generation): {avg_utoa_cached_call_time_per_game}s")

Average uci_to_action call time: 7.145404815673828e-07 s per call
Average uci_to_action call time per game (excluding train data generation): 0.2915325164794922 s
Average uci_to_action call time per game (including train data generation): 0.29367613792419434 s


In [12]:
# Calc time saved by caching uci to action
avg_utoa_time_diff = np.abs(avg_utoa_call_time_per_game - avg_utoa_cached_call_time_per_game)
print(f"Average uci_to_action call time difference: {avg_utoa_time_diff}s per {avg_moves_per_game} moves")
print(f"Average uci_to_action call time difference: {avg_utoa_time_diff * 100}s per 10000 training samples")
avg_time_diff_s = avg_utoa_time_diff * 5000
print(f"Average uci_to_action call time difference: {avg_time_diff_s}s per 500000 training samples")
avg_time_diff_m = avg_time_diff_s / 60
print(f"Average uci_to_action call time difference: {avg_time_diff_m}m per 500000 training samples")

Average uci_to_action call time difference: 43.21017622947693 s per 100 moves
Average uci_to_action call time difference: 4321.017622947693 s per 10000 training samples
Average uci_to_action call time difference: 216050.88114738464 s per 500000 training samples
Average uci_to_action call time difference: 3600.8480191230774 m per 500000 training samples


##### Testing optimization #1


In [14]:
from actionspace.maps import map_w, map_b
config = Config()
config.max_game_length = 200
times = []
while len(times) < 1000:
    game = Game(config)
    while not game.terminal():
        move = np.random.choice(game.legal_moves())
        start = time()
        action = map_w[move] if game.to_play() else map_b[move]
        end = time()
        game.make_move(move)
        times.append(end - start)
        if len(times) == 1000:
            break
avg_utoa_cached_call_time = np.mean(times)

In [15]:
num_utoa_calls = avg_moves_per_game * (num_simulations + 1) * avg_num_moves_per_position #  During MCTS on end of each simulation
print(f"Average uci_to_action call time: {avg_utoa_cached_call_time}s per call")
avg_utoa_cached_call_time_per_game = avg_utoa_cached_call_time * num_utoa_calls
print(f"Average uci_to_action call time per game (excluding train data generation): {avg_utoa_cached_call_time_per_game} s")
num_utoa_calls += avg_moves_per_game * avg_num_moves_per_position
avg_utoa_cached_call_time_per_game = avg_utoa_cached_call_time * num_utoa_calls
print(f"Average uci_to_action call time per game (including train data generation): {avg_utoa_cached_call_time_per_game} s")

Average uci_to_action call time: 9.558200836181641e-07 s per call
Average uci_to_action call time per game (excluding train data generation): 0.389974594116211 s
Average uci_to_action call time per game (including train data generation): 0.3928420543670655 s


In [16]:
# Calc time saved by caching uci to action
avg_utoa_time_diff = np.abs(avg_utoa_call_time_per_game - avg_utoa_cached_call_time_per_game)
print(f"Average uci_to_action call time difference: {avg_utoa_time_diff}s per {avg_moves_per_game} moves")
print(f"Average uci_to_action call time difference: {avg_utoa_time_diff * 100} s per 10000 training samples")
avg_time_diff_s = avg_utoa_time_diff * 5000
print(f"Average uci_to_action call time difference: {avg_time_diff_s}s per 500000 training samples")
avg_time_diff_m = avg_time_diff_s / 60
print(f"Average uci_to_action call time difference: {avg_time_diff_m}m per 500000 training samples")

Average uci_to_action call time difference: 43.11101031303406 s per 100 moves
Average uci_to_action call time difference: 4311.101031303406 s per 10000 training samples
Average uci_to_action call time difference: 215555.0515651703 s per 500000 training samples
Average uci_to_action call time difference: 3592.584192752838 m per 500000 training samples


##### Testing optimization in actual games (but with random values and policies)

###### Testing action space optimizations

In [5]:
def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    return value, policy

from mcts_old import run_mcts as run_mcts_old
from mcts import run_mcts as run_mcts_cached
from game import Game
from time import time
import numpy as np
from config import Config

config = Config()
config.max_game_length = 150
move_times_old = []
while len(move_times_old) < 1000:
    game = Game(config)
    while not game.terminal():
        start = time()
        move, _ = run_mcts_old(game, fake_network)
        end = time()
        move_times_old.append(end - start)
        if len(move_times_old) == 1000:
            break
        game.make_move(move)

move_times_cached = []
while len(move_times_cached) < 1000:
    game = Game(config)
    while not game.terminal():
        start = time()
        move, _ = run_mcts_cached(game, fake_network)
        end = time()
        move_times_cached.append(end - start)
        if len(move_times_cached) == 1000:
            break
        game.make_move(move)


In [16]:
# Data may still vary since games are different and simulations have different number of legal moves to evaluate
# Also dows not account for tree reuse, training data generation, etc.
print(f"Average move time old: {np.mean(move_times_old)}s")
print(f"Average move time cached: {np.mean(move_times_cached)}s")

avg_time_diff = np.abs(np.mean(move_times_old) - np.mean(move_times_cached))
print(f"Average move time difference: {avg_time_diff}s")

print(f"Total time old: {np.sum(move_times_old)}s")
print(f"Total time cached: {np.sum(move_times_cached)}s")

total_time_diff = np.abs(np.sum(move_times_old) - np.sum(move_times_cached))
print(f"Total time difference: {total_time_diff}s")

# Per a sample of 10000
print(f"Average move time difference: {avg_time_diff * 10000}s per 10000 samples")
print(f"Average move time difference (6 actors): {avg_time_diff * 10000 / 6}s per 10000 samples")
print(f"Average move time difference (6 actors) for a full training session: {((avg_time_diff * 500000) / 6) / 60 / 60}h per 500000 samples")



Average move time old: 0.8157329399585724s
Average move time cached: 0.5226809375286102s
Average move time difference: 0.2930520024299622s
Total time old: 815.7329399585724s
Total time cached: 522.6809375286102s
Total time difference: 293.05200242996216s
Average move time difference: 2930.520024299622s per 10000 samples
Average move time difference (6 actors): 488.420004049937s per 10000 samples
Average move time difference (6 actors) for a full training session: 6.783611167360236h per 500000 samples


###### Testing image updating optimizations

In [5]:
from actionspace.mapper import uci_to_action
from actionspace.maps import map_w, map_b

from mcts_old import run_mcts as run_mcts_old # Also uses old gameimage
from mcts import run_mcts as run_mcts
from game import Game
from time import time
import numpy as np
from config import Config

def calc_search_statistics_old(config: Config, root, to_play):
    sum_visits = sum([child.N for child in root.children.values()])
    child_visits = np.zeros(config.num_actions)
    for uci_move, child in root.children.items():
        action = uci_to_action(uci_move, to_play)
        child_visits[action] = child.N / sum_visits
    return child_visits

def calc_search_statistics(config: Config, root, to_play):
    sum_visits = sum([child.N for child in root.children.values()])
    child_visits = np.zeros(config.num_actions)
    for uci_move, child in root.children.items():
        action = map_w[uci_move] if to_play else map_b[uci_move]
        child_visits[action] = child.N / sum_visits
    return child_visits

def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    return value, policy


num_moves = 1000
 # Testing old implementation
config = Config()

config.max_game_length = 150

move_times_old = []
images, search_stats = [], []
while len(move_times_old) < num_moves:
    game = Game(config)
    while not game.terminal():
        start = time()
        move, root = run_mcts_old(game, fake_network, None, 100, 30)
        search_stats.append(calc_search_statistics_old(config, root, game.to_play()))
        images.append(game.make_image(-1))
        end = time()
        move_times_old.append(end - start)
        if len(move_times_old) == num_moves:
            break
        game.make_move(move)

# Testing new implementation with all optimizations
move_times_new = []
images, search_stats = [], []
while len(move_times_new) < num_moves:
    game = Game(config)
    while not game.terminal():
        start = time()
        move, root = run_mcts(config, game, fake_network, None, 100, 30)
        search_stats.append(calc_search_statistics(config, root, game.to_play()))
        images.append(root.image)
        end = time()
        move_times_new.append(end - start)
        if len(move_times_new) == num_moves:
            break
        game.make_move(move)


Average move time old: 0.8389750304222107s
Average move time new: 0.1310068609714508s


In [7]:
from actionspace import map_w, map_b
from mcts import run_mcts as run_mcts
from mcts.c import run_mcts as run_mcts_c
from game import Game
from time import time
import numpy as np
from config import Config
import tensorflow as tf

In [4]:
# Testing brand new implementation with cython and all optimizations
def calc_search_statistics(config: Config, root, to_play):
    sum_visits = sum([child.N for child in root.children.values()])
    child_visits = np.zeros(config.num_actions)
    for uci_move, child in root.children.items():
        action = map_w[uci_move] if to_play else map_b[uci_move]
        child_visits[action] = child.N / sum_visits
    return child_visits

def fake_network(images):
    value = np.random.rand()
    policy = np.random.rand(4672)
    return value, policy

config = Config()
config.max_game_length = 150
num_moves = 1000
move_times_new2 = []
images, search_stats = [], []
while len(move_times_new2) < num_moves:
    game = Game(config)
    root = None
    while not game.terminal():
        start = time()
        move, root = run_mcts(config, game, fake_network, root, False, 100, 30)
        search_stats.append(calc_search_statistics(config, root, game.to_play()))
        images.append(root.image)
        root = root.children[move]
        end = time()
        move_times_new2.append(end - start)
        if len(move_times_new2) == num_moves:
            break
        game.make_move(move)

In [5]:
print(f"Average move time (fully optimized): {np.mean(move_times_new2)}s")

Average move time (fully optimized): 0.12471763610839844s


In [16]:
# Testing brand new implementation with cython and all optimizations
def calc_search_statistics(config: Config, root, to_play):
    sum_visits = sum([child.N for child in root.children.values()])
    child_visits = np.zeros(config.num_actions)
    for uci_move, child in root.children.items():
        action = map_w[uci_move] if to_play else map_b[uci_move]
        child_visits[action] = child.N / sum_visits
    return child_visits

def fake_network(images):
    value = np.random.rand(1)
    policy = np.random.rand(4672)
    value = tf.convert_to_tensor(value, dtype=tf.float32)
    policy = tf.convert_to_tensor(policy, dtype=tf.float32)
    return {
        "value_head": [value],
        "policy_head": [policy]
    }

config = Config()
config.max_game_length = 150
num_moves = 1000
move_times_new2 = []
images, search_stats = [], []
while len(move_times_new2) < num_moves:
    game = Game()
    root = None
    while not game.terminal():
        start = time()
        move, root = run_mcts_c(config, game, fake_network, root, False, 100, 30)
        search_stats.append(calc_search_statistics(config, root, game.to_play()))
        images.append(root.image)
        root = root.children[move]
        end = time()
        move_times_new2.append(end - start)
        if len(move_times_new2) == num_moves:
            break
        game.make_move(move)

In [4]:
# With old onehot encode
print(f"Average move time (fully optimized): {np.mean(move_times_new2)}s")

Average move time (fully optimized): 0.08124334120750427s


In [5]:
# With new one hot encode
print(f"Average move time (fully optimized): {np.mean(move_times_new2)}s")

Average move time (fully optimized): 0.07955593657493591s


In [3]:
print(f"Average move time (fully optimized2): {np.mean(move_times_new2)}s")

Average move time (fully optimized2): 0.07532040333747864s


In [18]:
print(f"Average move time (fully optimized2): {np.mean(move_times_new2)}s")

Average move time (fully optimized2): 0.06751813793182374s


In [1]:
from actionspace import map_w, map_b
from mcts import run_mcts as run_mcts
from mcts.c import run_mcts as run_mcts_c
import cppchess
import chess
from game import Game
from time import time
import numpy as np
from config import Config
# Testing with a real model with actual predictions
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], True)

model = tf.saved_model.load("train/checkpoints/trt/14-02-2024_08:31:11/saved_model")
trt_func = model.signatures["serving_default"]

2024-02-17 12:36:24.481066: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-17 12:36:25.891311: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 905 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5


In [2]:
def calc_search_statistics(config: Config, root, to_play):
    sum_visits = sum([child.N for child in root.children.values()])
    child_visits = np.zeros(config.num_actions)
    for uci_move, child in root.children.items():
        action = map_w[uci_move] if to_play else map_b[uci_move]
        child_visits[action] = child.N / sum_visits
    return child_visits

config = Config()
config.max_game_length = 150
num_moves = 100

In [10]:
move_times_new = []
images, search_stats = [], []
while len(move_times_new) < num_moves:
    board = chess.Board()
    game = Game(board)
    root = None
    while not game.terminal():
        start = time()
        move, root = run_mcts(config, game, trt_func, root, False, 600, 30)
        search_stats.append(calc_search_statistics(config, root, game.to_play()))
        images.append(root.image)
        root = root.children[move]
        end = time()
        move_times_new.append(end - start)
        if len(move_times_new) == num_moves:
            break
        game.make_move(move)

In [11]:
print("TIMES FOR ACTUAL PREDICTIONS WITHOUT CYTHON OR C++")
print(f"Average move time (fully optimized): {np.mean(move_times_new)}s")
print(f"Total time (fully optimized): {np.sum(move_times_new)}s for {len(images)} moves")

TIMES FOR ACTUAL PREDICTIONS WITHOUT CYTHON OR C++
Average move time (fully optimized): 0.8924691820144653s
Total time (fully optimized): 89.24691820144653s for 100 moves


In [12]:
move_times_new = []
images, search_stats = [], []
while len(move_times_new) < num_moves:
    board = cppchess.Board()
    game = Game(board)
    root = None
    while not game.terminal():
        start = time()
        move, root = run_mcts_c(config, game, trt_func, root, False, 600, 30)
        search_stats.append(calc_search_statistics(config, root, game.to_play()))
        images.append(root.image)
        root = root.children[move]
        end = time()
        move_times_new.append(end - start)
        if len(move_times_new) == num_moves:
            break
        game.make_move(move)

In [13]:
print("TIMES FOR ACTUAL PREDICTIONS WITH CYTHON & C++")
print(f"Average move time (fully optimized): {np.mean(move_times_new)}s")
print(f"Total time (fully optimized): {np.sum(move_times_new)}s for {len(images)} moves")

TIMES FOR ACTUAL PREDICTIONS WITH CYTHON & C++
Average move time (fully optimized): 0.9618119263648987s
Total time (fully optimized): 96.18119263648987s for 100 moves


In [1]:
from config import Config
from game import Game
import numpy as np
from time import time
import chess

2024-02-12 11:28:39.488085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from gameimage_old import board_to_image
from gameimage import board_to_image as board_to_image_new
num_times = 1000
config = Config()
config.max_game_length = 200
times1 = []
times2 = []
run = True
while run:
    game = Game(config)
    while not game.terminal():
        # Test prev impl
        start = time()
        img1 = board_to_image(game.board)
        end = time()
        times1.append(end - start)

        # Test new impl
        start = time()
        img2 = board_to_image_new(game.board)
        end = time()
        times2.append(end - start)

        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times2) == num_times:
            run = False
            break

In [3]:
print(f"Average image generation time (old): {np.mean(times1)}s per image")
print(f"Average image generation time (new): {np.mean(times2)}s per image")

Average image generation time (old): 0.004028122901916504s per image
Average image generation time (new): 0.00089847731590271s per image


In [6]:
from gameimage_old import board_to_image, update_image
from gameimage import board_to_image as board_to_image_new
from gameimage import update_image as update_image_new

num_times = 1000
config = Config()
config.max_game_length = 200
times1 = []
times2 = []
run = True
while run:
    game = Game(config)
    img1, img2 = None, None
    while not game.terminal():
        # Test prev impl
        start = time()
        if img1 is None:
            img1 = board_to_image(game.board)
        else:
            img1 = update_image(game.board, img1)
        end = time()
        times1.append(end - start)

        # Test new impl
        start = time()
        if img2 is None:
            img2 = board_to_image_new(game.board)
        else:
            img2 = update_image_new(game.board, img2)
        end = time()
        times2.append(end - start)

        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times2) == num_times:
            run = False
            break

In [7]:
print(f"Average image generation / update time (old): {np.mean(times1)}s per image")
print(f"Average image generation / update time (new): {np.mean(times2)}s per image")

Average image generation / update time (old): 0.000598430871963501s per image
Average image generation / update time (new): 0.00014314079284667968s per image


In [3]:
from config import Config
from game import Game
import numpy as np
from time import time
import cppchess.cychess as chess
from gameimage.c import board_to_image, update_image

num_times = 1000
config = Config()
config.max_game_length = 200
times1 = []
run = True
while run:
    game = Game()
    img = None
    while not game.terminal():
        # Test prev impl
        start = time()
        if img is None:
            img = board_to_image(game.board)
        else:
            img = update_image(game.board, img)
        end = time()
        times1.append(end - start)
        move = np.random.choice(game.legal_moves())
        game.make_move(move)
        if len(times1) == num_times:
            run = False
            break


In [4]:
print(f"Average image generation / update time (cpp): {np.mean(times1)}s per image")

Average image generation / update time (cpp): 8.166742324829102e-05s per image
